In [13]:
#import libraries

import tensorflow as tf
import tensorrt as trt
from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization, Dropout
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import pandas as pd
import import_ipynb
from skimage.io import imread, imshow
from skimage.transform import resize
import os
from zipfile import ZipFile


In [14]:
# import dataset

import opendatasets as od

od.download("https://www.kaggle.com/datasets/quadeer15sh/augmented-forest-segmentation")

Skipping, found downloaded files in "./augmented-forest-segmentation" (use force=True to force download)


In [16]:
# segmentation classification model class

class SegmentationClassificationModel: 
    
    #initializes object with file path to the csv file, number of classes, folder to the dataset, csv image column, csv mask column, image height, image width, and image channels
    def __init__(self, datasetCSVFilePath, numClasses, datasetFolderPath, X_col, y_col, img_height, img_width, img_channels):
        self.datasetCSVFilePath = datasetCSVFilePath
        self.datasetFolderPath = datasetFolderPath
        self.X_col = X_col
        self.y_col = y_col
        self.img_height = img_height
        self.img_width = img_width
        self.img_channels = img_channels
        self.numClasses = numClasses
        
    # 
    def getDataset(self):
        
        #put csv in dataframe
        myDatasetCSV = pd.read_csv(self.datasetCSVFilePath)
        
        # create two numpy arrays to put the image and mask
        self.X = np.zeros((len(myDatasetCSV), self.img_height, self.img_width, self.img_channels), dtype = np.uint8)
        self.y = np.zeros((len(myDatasetCSV), self.img_height, self.img_width, 1), dtype = bool)
        i = 0
        
        # for each index and row in the csv file, read the image into a numpy array, resize it, and put it in X numpy array
        for index, row in myDatasetCSV.iterrows():
            
            imagePath = self.datasetFolderPath + "/images/" + row[self.X_col]
            myImage = imread(imagePath)[:,:,:self.img_channels]  
            myImage = resize(myImage, (self.img_height, self.img_width), mode='constant', preserve_range=True)
            self.X[i] = myImage
            
            # Load mask 
            maskDir = os.path.join(self.datasetFolderPath, "masks")
            maskArray = np.zeros((self.img_height, self.img_width, 1), dtype=bool)
            for mask_file in os.listdir(maskDir):
                if mask_file.startswith(row[self.y_col]):
                    maskPath = os.path.join(maskDir, mask_file)
                    myMask = imread(maskPath)[:, :, 0]  # Assuming grayscale
                    myMask = np.expand_dims(resize(myMask, (self.img_height, self.img_width), mode='constant',  
                                        preserve_range=True), axis=-1)
                    maskArray = np.maximum(maskArray, myMask)
        
        self.y[i] = maskArray
        i += 1
    
    #split dataset into train and test 
    def trainTestSplit(self):
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = 0.2)
    
    # divide by 255 for relu activate do that the pixel range is from 0 - 1 rather than 0 - 255
    def preProcessing(self):
        
        self.X_train = self.X_train / 255.0
        self.X_test = self.X_test / 255.0
    
    # 
    def trainValSplit(self):
        
        self.X_train, X_val, self.y_train, y_val = train_test_split(self.X_train, self.y_train, test_size = 0.2)
        
        self.X_val = X_val
        self.y_val = y_val
    
    # was used for the cnn model to display images of the 
    # def displayClassImages(self):
        
    #     plt.figure(figsize=(10,10))
    #     for i in range(6):
    #         plt.subplot(5,5,i+1)
    #         plt.xticks([])
    #         plt.yticks([])
    #         plt.grid(False)
    #         plt.imshow(self.X_train[i])
    #         plt.xlabel(self.y_train[i])
    #     plt.show()
            
    def augmentDataset(self, h_flip, v_flip, rot_range, myZoom_range):
        
        image_generator = ImageDataGenerator(
            horizontal_flip = h_flip,
            vertical_flip = v_flip,
            rotation_range = rot_range,
            zoom_range = myZoom_range)
        
        self.train_generator = image_generator.flow(self.X_train, self.y_train)
    
    # cnn model function used for testing earlier
    
    # def cnnModel(self, numConvolutions, numDropout, dropoutRate, isBatchNormalization):
        
    #     myNumDropout = numDropout
        
    #     self.model = models.Sequential()
    #     self.model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(self.img_height, self.img_width, self.img_channels)))
    #     self.model.add(layers.MaxPooling2D((2, 2)))
        
    #     for i in range(0, numConvolutions):
            
    #         self.model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    #         self.model.add(layers.MaxPooling2D((2, 2)))
            
    #         if(myNumDropout > 0):
                
    #             self.model.add(Dropout(dropoutRate))
    #             myNumDropout -= 1
                
    #         if(isBatchNormalization == True):
                
    #             self.model.add(BatchNormalization())
        
    #     self.model.add(layers.Flatten())
    #     self.model.add(layers.Dense(64, activation='relu'))
    #     self.model.add(layers.Dense(self.num_classes))
    
    # u-net model with a dropout rate input, contracts and then expands the 
    def uNetModel(self, dropoutRate):
        
        self.inputs = tf.keras.layers.Input((self.img_height, self.img_width, self.img_channels))
        
        #Contraction path
        conv1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(self.inputs)
        conv1 = tf.keras.layers.Dropout(dropoutRate)(conv1)
        conv1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
        batch1 = tf.keras.layers.BatchNormalization()(conv1)
        relu1 = tf.keras.layers.ReLU()(batch1)
        pool1 = tf.keras.layers.MaxPooling2D((2, 2))(relu1)

        conv2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
        conv2 = tf.keras.layers.Dropout(dropoutRate)(conv2)
        conv2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
        batch2 = tf.keras.layers.BatchNormalization()(conv2)
        relu2 = tf.keras.layers.ReLU()(batch2)
        pool2 = tf.keras.layers.MaxPooling2D((2, 2))(relu2)
        
        conv3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
        conv3 = tf.keras.layers.Dropout(dropoutRate)(conv3)
        conv3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
        batch3 = tf.keras.layers.BatchNormalization()(conv3)
        relu3 = tf.keras.layers.ReLU()(batch3)
        pool3 = tf.keras.layers.MaxPooling2D((2, 2))(relu3)
        
        conv4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
        conv4 = tf.keras.layers.Dropout(dropoutRate)(conv4)
        conv4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
        batch4 = tf.keras.layers.BatchNormalization()(conv4)
        relu4 = tf.keras.layers.ReLU()(batch4)
        pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(relu4)
        
        conv5 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
        batch5 = tf.keras.layers.BatchNormalization()(conv5)
        relu5 = tf.keras.layers.ReLU()(batch5)
        conv5 = tf.keras.layers.Dropout(dropoutRate)(relu5)
        conv5 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)

        # Expansive path 
        up1 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv5)
        up1 = tf.keras.layers.concatenate([up1, conv4])
        up1 = tf.keras.layers.BatchNormalization()(up1)
        up1 = tf.keras.layers.ReLU()(up1)

        
        up2 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(up1)
        up2 = tf.keras.layers.concatenate([up2, conv3])
        up2 = tf.keras.layers.BatchNormalization()(up2)
        up2 = tf.keras.layers.ReLU()(up2)

        
        up3 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(up2)
        up3 = tf.keras.layers.concatenate([up3, conv2])
        up3 = tf.keras.layers.BatchNormalization()(up3)
        up3 = tf.keras.layers.ReLU()(up3)
        
        up4 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(up3)
        up4 = tf.keras.layers.concatenate([up4, conv1], axis=3)
        up4 = tf.keras.layers.BatchNormalization()(up4)
        up4 = tf.keras.layers.ReLU()(up4)
        
        self.outputs = tf.keras.layers.Conv2D(self.numClasses, (1, 1), activation='sigmoid')(up4)

        
    # compiles and fits the cnn model (adam optimizer)
    # def compileAndFitCNNModel(self, learningRate, numEpochs):
        
    #     optimizer = tf.keras.optimizers.Adam(learning_rate = learningRate)
        
        
    #     self.model.compile(optimizer= optimizer,
    #           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)),
    
    #           metrics=['accuracy'])

    #     self.history = self.model.fit(self.train_generator, epochs=numEpochs,
    #                         validation_data=(self.X_val, self.y_val))
        
    #compiles and fits u-net model (adam optimizer) (binary crossentropy)
    def compileAndFitUNetModel(self, learningRate, numEpochs, batchSize):
        
        self.model = tf.keras.Model(inputs=[self.inputs], outputs=[self.outputs])
        
        optimizer = tf.keras.optimizers.Adam(learning_rate = learningRate)
        
        self.model.compile(optimizer= optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

        self.history = self.model.fit(self.train_generator, epochs=numEpochs, batch_size = batchSize,
                            validation_data=(self.X_val, self.y_val)) 
    
    def showModelLoss(self):
        
        #graph of the loss function between the training data and the validiation data

        plt.figure(figsize=(15, 16))

        plt.subplot(4, 2, 2)
        plt.plot(self.history.history['loss'], label='Loss')
        plt.plot(self.history.history['val_loss'], label='val_Loss')
        plt.title('Loss')
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        
    def showModelAccuracy(self):    
        
        #graph of the accuracy between the training data and the validiation data

        plt.figure(figsize=(15, 16))

        plt.subplot(4, 2, 2)
        plt.plot(self.history.history['accuracy'], label='accuracy')
        plt.plot(self.history.history['val_accuracy'], label='val_accuracy')
        plt.title('Accuracy')
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.legend()
    

    def showTable(self):
        
        # Extract accuracy, loss, validation accuracy, and validation loss values from the history object
        history_dict = self.history.history
        history_dict['epoch'] = range(1, len(history_dict['accuracy']) + 1)

        # Convert the history dictionary to a DataFrame
        self.historyDF = pd.DataFrame(history_dict)
        
        for key in history_dict.keys():
            if key != 'epoch':
                history_dict[key] = [round(value, 3) for value in history_dict[key]]

        # Create a Matplotlib figure and axis
        fig, ax = plt.subplots(figsize=(8, 6))

        # Hide axes
        ax.axis('off')
        plt.title("Accuracy and Loss over Epochs")

        # Create the table
        ax.table(cellText=self.historyDF.values,
                 colLabels=self.historyDF.columns,
                 cellLoc='center',
                 loc='center')

        # Show the plot
        plt.show()
    